## Setup and Imports

In [ ]:
!pip install -Uq diffusers
!pip install -Uq transformers
!pip install -Uq bitsandbytes

In [ ]:
from transformers import T5EncoderModel, CLIPTextModelWithProjection
from transformers import BitsAndBytesConfig as TransformersBitsAndBytesConfig

from diffusers import SD3Transformer2DModel, AutoencoderKL
from diffusers import BitsAndBytesConfig as DiffusersBitsAndBytesConfig
import torch

# Text Encoders

In [ ]:
model_id = "stabilityai/stable-diffusion-3-medium-diffusers"

nf4_config = TransformersBitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

text_encoder = CLIPTextModelWithProjection.from_pretrained(
    model_id, subfolder="text_encoder",
    quantization_config=nf4_config,
    torch_dtype=torch.float16
)

text_encoder_2 = CLIPTextModelWithProjection.from_pretrained(
    model_id, subfolder="text_encoder_2",
    quantization_config=nf4_config,
    torch_dtype=torch.float16
)

text_encoder_3 = T5EncoderModel.from_pretrained(
    model_id, subfolder="text_encoder_3",
    quantization_config=nf4_config,
    torch_dtype=torch.float16
)

In [ ]:
text_encoder.save_pretrained("text_encoder")
text_encoder_2.save_pretrained("text_encoder_2")
text_encoder_3.save_pretrained("text_encoder_3")

## Diffusion Transformers

In [ ]:
model_id = "stabilityai/stable-diffusion-3-medium-diffusers"

nf4_config = DiffusersBitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

transformer = SD3Transformer2DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=nf4_config,
    torch_dtype=torch.float16
)
vae = AutoencoderKL.from_pretrained(
    model_id,
    subfolder="vae",
    quantization_config=nf4_config,
    torch_dtype=torch.float16
)

In [ ]:
transformer.save_pretrained("transformer")
vae.save_pretrained("vae")

## Upload to the Hub

In [ ]:
from huggingface_hub import create_repo, upload_folder

repo_id = create_repo(repo_id="ariG23498/sd3-medium-nf4-pkg", private=True, repo_type="model").repo_id

upload_folder(repo_id=repo_id, folder_path="text_encoder", path_in_repo="text_encoder")
upload_folder(repo_id=repo_id, folder_path="text_encoder_2", path_in_repo="text_encoder_2")
upload_folder(repo_id=repo_id, folder_path="text_encoder_3", path_in_repo="text_encoder_3")

upload_folder(repo_id=repo_id, folder_path="transformer", path_in_repo="transformer")
upload_folder(repo_id=repo_id, folder_path="vae", path_in_repo="vae")